In [4]:
import json, codecs, os
import sqlite3

In [5]:
nodes_fpath = os.getcwd() + '/raw_graph_schema/node_properties.json'
rels_fpath = os.getcwd() + '/raw_graph_schema/rel_properties.json'

In [6]:

db_fpath =  '/home/22842219/Desktop/projects_data/spider/database'
sorted_schema = {} # {'db_id': {'a':[], 'b':[]}}
for fpath in [nodes_fpath, rels_fpath]:
    with codecs.open(fpath, 'r', 'utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if 'nodeType' in every:
                tag = every['nodeType']
            elif 'relType' in every:
                tag = every['relType']
            tag_seps = tag.strip(':').strip("`").split('.')
            db_id = tag_seps[0]
            if db_id not in sorted_schema:
                sorted_schema[db_id]={}
                sorted_schema[db_id]['property_types']=[]
            if tag not in sorted_schema[db_id]:
                sorted_schema[db_id][tag]= []

            if isinstance(every['propertyTypes'], list) and len(every['propertyTypes'])==1:
                sorted_schema[db_id]['property_types'].append('_'.join(every['propertyTypes']))
            else:
                sorted_schema[db_id]['property_types'].append(every['propertyTypes'])
                
            # use sqlite cursor to get field values. 
            # we use the concept of picklist to save field values.
            if db_id == 'department_management':
                tag_name = tag_seps[1]
                db_path = os.path.join(db_fpath, db_id, '{}.sqlite'.format(db_id))
                if every['propertyName']:
                    fetch_sql = "SELECT DISTINCT `{}` FROM `{}`".format(every['propertyName'], tag_name)
                    print(db_id, tag_name, fetch_sql)
                    try:
                        conn = sqlite3.connect(db_path)
                        conn.text_factory = bytes
                        c = conn.cursor()
                        c.execute(fetch_sql)
                        picklist = set()
                        # print(c.fetchall())
                        for x in c.fetchall():
                            if isinstance(x[0], str):
                                picklist.add(x[0].encode("utf-8"))
                            elif isinstance(x[0], bytes):
                                try:
                                    picklist.add(x[0].decode("utf-8"))
                                except UnicodeDecodeError:
                                    picklist.add(x[0].decode("latin-1"))
                            else:
                                picklist.add(x[0])
                        picklist = list(picklist)
                    finally:
                        conn.close()
                    sorted_schema[db_id][tag].append({every['propertyName']: picklist})



department_management department SELECT DISTINCT `Name` FROM `department`
department_management department SELECT DISTINCT `Creation` FROM `department`
department_management department SELECT DISTINCT `Department_ID` FROM `department`
department_management department SELECT DISTINCT `Ranking` FROM `department`
department_management department SELECT DISTINCT `Budget_in_Billions` FROM `department`
department_management department SELECT DISTINCT `Num_Employees` FROM `department`
department_management head SELECT DISTINCT `name` FROM `head`
department_management head SELECT DISTINCT `born_state` FROM `head`
department_management head SELECT DISTINCT `age` FROM `head`
department_management head SELECT DISTINCT `head_ID` FROM `head`
department_management management SELECT DISTINCT `temporary_acting` FROM `management`


In [7]:
sorted_schema['department_management']


{'property_types': ['String',
  'String',
  'Long',
  'Long',
  'Double',
  'Double',
  'String',
  'String',
  'Double',
  'Long',
  'String'],
 ':`department_management.department`': [{'Name': ['Labor',
    'State',
    'Agriculture',
    'Education',
    'Defense',
    'Interior',
    'Commerce',
    'Transportation',
    'Energy',
    'Treasury',
    'Justice',
    'Homeland Security',
    'Health and Human Services',
    'Housing and Urban Development',
    'Veterans Affairs']},
  {'Creation': ['1977',
    '1979',
    '2002',
    '1913',
    '1849',
    '1870',
    '1789',
    '1903',
    '1966',
    '1889',
    '1947',
    '1953',
    '1989',
    '1965']},
  {'Department_ID': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]},
  {'Ranking': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]},
  {'Budget_in_Billions': [439.3,
    6.2,
    9.96,
    10.7,
    11.1,
    73.2,
    77.6,
    46.2,
    44.6,
    21.5,
    23.4,
    58.0,
    59.7,
    62.8,
    543.2]},
  {'Num_Empl

In [5]:
import os
outfpath= os.getcwd() +  '/schema.json'
with open(outfpath, 'w') as out:
    json.dump(sorted_schema, out, indent=4)

In [6]:
sp_data_folder =  '/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder'
for split in ['train.json', 'dev.json']:
    fpth = os.path.join(sp_data_folder,  split )
    print(fpth)
    reformated_data = []
    with open(fpth, 'r', encoding='utf-8-sig') as f:
        data = json.load(f)
        for every in data:
            if every['answers'] and every['answers']!=[[0]]:
                reformated = {}
                reformated['question'] = every['question']
                reformated['query'] = every['cypher_query']
                reformated['db_id'] = every['db_id']
                reformated['answers'] = every['answers']
                reformated_data.append(reformated)

        with open(split, 'w') as out:
            json.dump(reformated_data, out, indent=4)

                    


/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder/train.json
/home/22842219/Desktop/phd/SemanticParser4Graph/sp_data_folder/dev.json
